In [1]:
from pathlib import Path

import pandas as pd
import verifiers as vf
from dotenv import load_dotenv
from openai import AsyncOpenAI


assert load_dotenv(), "Failed to load .env file"

In [ ]:
from rlvr.utils.llm import get_default_model

env_id = "vf-musique-structured"
datasets_str = "bdsaglam/musique-mini,answerable,validation"
noise_rate = 0.0
retriever = "hybrid"
model = get_default_model()

params = {
    "env_id": env_id,
    "datasets_str": datasets_str,
    "noise_rate": noise_rate,
    "retriever": retriever,
    "model": model,
    "sampling_args": {
        "temperature": 0.5,
        "max_tokens": 1024, # max new tokens to generate
        # "top_p": 0.95,
        # "min_p": None,
    },
    "max_concurrent": 16,
}
params

{'env_id': 'vf-musique-structured',
 'datasets_str': 'bdsaglam/musique-mini,answerable,validation',
 'noise_rate': 0.0,
 'retriever': 'hybrid',
 'model': 'Qwen/Qwen2.5-7B-Instruct',
 'sampling_args': {'temperature': 0.5, 'max_tokens': 1024},
 'max_concurrent': 8}

In [3]:
# import mlflow

# # Tell MLflow about the server URI.
# mlflow.set_tracking_uri("http://127.0.0.1:5000")
# # Enable autologging with all features
# mlflow.openai.autolog()
# # Create a unique name for your experiment.
# mlflow.set_experiment("rlvr-eval")

In [4]:
# Load MuSiQue environment
print("🌍 Loading MuSiQue environment...")

vf_env = vf.load_environment(
    env_id=env_id,
    datasets_str=datasets_str,
    noise_rate=noise_rate,
    retriever=retriever,
)
print(f"✅ Environment loaded with {len(vf_env.dataset)} examples")

# Use OpenAI-compatible API client (e.g., for vLLM)
print("🤖 Using OpenAI-compatible API client...")

# Run evaluation using the environment
print("🔄 Running evaluation...")
openai_client = AsyncOpenAI()
results = vf_env.evaluate(
    openai_client,
    model,
    rollouts_per_example=1,
    sampling_args=params["sampling_args"],
    max_concurrent=params["max_concurrent"],
)


2025-09-26 19:00:27 - verifiers.utils.env_utils - INFO - Loading environment: vf-musique-structured
2025-09-26 19:00:27 - verifiers.utils.env_utils - INFO - Using provided args: noise_rate=0.0, retriever=hybrid, datasets_str=bdsaglam/musique-mini,answerable,validation
2025-09-26 19:00:27 - verifiers.utils.env_utils - INFO - Using default args: eval_datasets_str=None


🌍 Loading MuSiQue environment...


Map: 100%|##########| 300/300 [00:00<?, ? examples/s]

Filter:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

2025-09-26 19:00:31 - verifiers.utils.env_utils - INFO - Successfully loaded environment 'vf-musique-structured'
2025-09-26 19:00:31 - verifiers.envs.MuSiQueEnv - INFO - eval_dataset is not set, falling back to train dataset


✅ Environment loaded with 100 examples
🤖 Using OpenAI-compatible API client...
🔄 Running evaluation...


Running 100 rollouts (interleaved): 100%|██████████| 100/100 [00:57<00:00,  1.74it/s]


In [5]:
ds = vf_env.make_dataset(results)
df = ds.to_pandas()

In [6]:
df.describe()

,reward,exact_match_reward,f1_reward,retrieval_recall_reward,retrieval_precision_reward,citation_reward,format_reward,combined_reward
count,100.000000,100.000000,100.000000,100.000000,100.0,100.000000,100.000000,100.000000
mean,0.633868,0.400000,0.533858,0.795000,1.0,0.610000,0.940000,0.633868
std,0.252353,0.492366,0.454602,0.247156,0.0,0.280512,0.238683,0.252353
min,0.225000,0.000000,0.000000,0.500000,1.0,0.000000,0.000000,0.225000
25%,0.428125,0.000000,0.000000,0.500000,1.0,0.500000,1.000000,0.428125
50%,0.562500,0.000000,0.666667,1.000000,1.0,0.500000,1.000000,0.562500
75%,0.925000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,0.925000
max,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000


In [7]:
from datetime import datetime
import json

exp_id = datetime.now().strftime("%Y%m%d_%H%M%S")
exp_dir = Path(f"../outputs/musique-eval/{exp_id}")
exp_dir.mkdir(parents=True, exist_ok=True)

(exp_dir / "params.json").write_text(json.dumps(params, indent=2))
ds.to_json(exp_dir / "musique-eval-results.jsonl", orient="records", lines=True)
df.describe().to_csv(exp_dir / "scores.csv")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [8]:
failed_df = df[df['f1_reward'] < 0.1]
len(failed_df)

37